In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import scipy.io as sio
from keras.utils import np_utils

%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, cohen_kappa_score
from sklearn.decomposition import PCA

from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Reshape, Dropout
from keras.layers.normalization import BatchNormalization
from keras import optimizers

from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping

import cv2
from skimage.util import view_as_windows

In [2]:
from config.utils import *
from config.models import *

In [72]:
######### GLOBAL PARAMETERS #########
dataset = 'PU'
testSize = 0.85
windowSize = 25
K = 30 if dataset == 'IP' else 15
EPOCHS = 20
######### GLOBAL PARAMETERS #########

In [73]:
X, Y = loadData(dataset)

In [74]:
units, target_names = targets(dataset)

In [75]:
X.shape

(610, 340, 103)

In [76]:
Y.shape

(610, 340)

In [77]:
X,pca = applyPCA(X,N=K)
ev=pca.explained_variance_ratio_
print('X.shape',X.shape)
print('y.shape',Y.shape)

X.shape (610, 340, 15)
y.shape (610, 340)


In [78]:
X , Y = createImageCubes(X,Y,windowSize=windowSize)

In [79]:
print('X.shape',X.shape)
print('y.shape',Y.shape)

X.shape (42776, 25, 25, 15)
y.shape (42776,)


In [80]:
X_Train , X_Test , Y_Train, Y_Test = train_test_split(X,Y,test_size=testSize,random_state=345)

In [81]:
Y_Train = np_utils.to_categorical(Y_Train)
Y_Train.shape

(6416, 9)

In [82]:
X_Train = X_Train.reshape(-1, windowSize, windowSize, K, 1)
X_Train.shape

(6416, 25, 25, 15, 1)

In [83]:
X_Test = X_Test.reshape(-1, windowSize, windowSize, K, 1)
X_Test.shape

(36360, 25, 25, 15, 1)

In [84]:
Y_Test = np_utils.to_categorical(Y_Test)
Y_Test.shape

(36360, 9)

In [85]:
X_Train.shape

(6416, 25, 25, 15, 1)

In [24]:
Model = make_3d_1(units, dataset)

In [25]:
Model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_2 (Conv3D)            (None, 23, 23, 11, 8)     368       
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 21, 21, 9, 16)     3472      
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 10, 10, 4, 16)     0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 10, 10, 4, 16)     64        
_________________________________________________________________
reshape_1 (Reshape)          (None, 10, 10, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 8, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 4096)             

In [26]:
adam = Adam(lr=0.001, decay=1e-06)
Model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

filepath = "first" + dataset + ".hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='accuracy', verbose=1, save_best_only=True, mode='max')
early_stop = EarlyStopping(monitor='val_loss',patience=2)

callbacks_list = [checkpoint]

In [1]:
History = Model.fit(x=X_Train, y=Y_Train,epochs=EPOCHS, batch_size = 128, callbacks = callbacks_list)

In [29]:
Y_Pred = Model.predict(X_Test)
Y_Pred = np.argmax(Y_Pred, axis=1)

In [30]:
Report = (classification_report(np.argmax(Y_Test, axis=1), Y_Pred, target_names=target_names))
print(Report)

Matrix = (confusion_matrix(np.argmax(Y_Test, axis=1), Y_Pred))

                      precision    recall  f1-score   support

             Asphalt       0.98      0.65      0.78      5964
             Meadows       1.00      0.98      0.99     16779
              Gravel       0.95      0.90      0.92      1899
               Trees       0.96      0.97      0.97      2772
Painted metal sheets       0.99      1.00      0.99      1218
           Bare Soil       1.00      1.00      1.00      4500
             Bitumen       1.00      0.88      0.93      1193
Self-Blocking Bricks       0.63      0.99      0.77      3324
             Shadows       0.56      0.99      0.72       850

            accuracy                           0.93     38499
           macro avg       0.90      0.93      0.90     38499
        weighted avg       0.95      0.93      0.93     38499



In [86]:
Model = make_3d_2(units, dataset)

In [87]:
Model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_20 (Conv3D)           (None, 23, 23, 9, 8)      512       
_________________________________________________________________
conv3d_21 (Conv3D)           (None, 21, 21, 5, 16)     5776      
_________________________________________________________________
conv3d_22 (Conv3D)           (None, 19, 19, 3, 16)     6928      
_________________________________________________________________
conv3d_23 (Conv3D)           (None, 17, 17, 1, 16)     6928      
_________________________________________________________________
batch_normalization_6 (Batch (None, 17, 17, 1, 16)     64        
_________________________________________________________________
reshape_6 (Reshape)          (None, 17, 17, 16)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 15, 15, 64)       

In [88]:
adam = Adam(lr=0.001, decay=1e-06)
Model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

filepath = "second" + dataset + ".hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='accuracy', verbose=1, save_best_only=True, mode='max')
early_stop = EarlyStopping(monitor='val_loss',patience=2)

callbacks_list = [checkpoint]

In [2]:
History = Model.fit(x=X_Train, y=Y_Train,epochs=EPOCHS, batch_size = 128, callbacks = callbacks_list)

In [70]:
Y_Pred = Model.predict(X_Test)
Y_Pred = np.argmax(Y_Pred, axis=1)

In [71]:
Report = (classification_report(np.argmax(Y_Test, axis=1), Y_Pred, target_names=target_names))
print(Report)

Matrix = (confusion_matrix(np.argmax(Y_Test, axis=1), Y_Pred))

                      precision    recall  f1-score   support

             Asphalt       0.99      0.64      0.78      5964
             Meadows       1.00      1.00      1.00     16779
              Gravel       0.64      0.95      0.76      1899
               Trees       0.99      0.97      0.98      2772
Painted metal sheets       0.99      1.00      0.99      1218
           Bare Soil       0.89      1.00      0.94      4500
             Bitumen       0.91      0.96      0.93      1193
Self-Blocking Bricks       0.73      0.79      0.76      3324
             Shadows       0.58      0.82      0.68       850

            accuracy                           0.91     38499
           macro avg       0.86      0.90      0.87     38499
        weighted avg       0.93      0.91      0.91     38499

